In [32]:
!pip install -qU langchain pypdf langchain-community langchain-core langchain-openai langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00


In [33]:
from langchain_community.document_loaders import PyPDFLoader
def pdf_loader(file_path):
  """
    Loads a PDF from the given file path and returns its pages as a list.

    Args:
        file_path (str): Path to the PDF file.

    Returns:
        list: A list of page objects representing the pages in the PDF.

    Example:
        pdf_pages = pdf_loader("example.pdf")
  """
  loader = PyPDFLoader(file_path)
  pages = []
  for page in loader.load():
      pages.append(page)
  return pages

In [24]:
pages = pdf_loader("/content/handbook.pdf")

In [25]:
pages[0]

Document(metadata={'source': '/content/handbook.pdf', 'page': 0}, page_content='Zania, Inc.\nZania Employee Handbook\nSeptember 07, 2023')

In [15]:
import os
os.environ["OPENAI_API_KEY"] = ""


In [35]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


In [85]:
from pydantic import BaseModel, Field
from langchain_core.tools import tool, BaseTool
from typing import Optional, Type, List, Union, Any

from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

from langchain_openai import ChatOpenAI

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini")

# Function to load the PDF content
def pdf_loader(file_path):
  """
    Loads a PDF from the given file path and returns its pages as a list.

    Args:
        file_path (str): Path to the PDF file.

    Returns:
        list: A list of page objects representing the pages in the PDF.

    Example:
        pdf_pages = pdf_loader("example.pdf")
  """
  loader = PyPDFLoader(file_path)
  pages = []
  for page in loader.load():
      pages.append(page)
  return pages

# Function to create a vector store from loaded pages
def create_vector_store(pages):
  """
    Creates an in-memory vector store from the provided pages.

    Args:
        pages (list): A list of page objects to be indexed.

    Returns:
        InMemoryVectorStore: The created vector store containing the indexed pages.
  """
  vector_store = InMemoryVectorStore.from_documents(pages, OpenAIEmbeddings())
  return vector_store

# Simplified UserInput model
class UserInput(BaseModel):
    """
    Pydantic model to define the input data for the retrieval QA system.

    Attributes:
        file_path (str): Path to the local PDF file.
        questions (List[str]): List of questions to ask.
    """
    file_path: str = Field(..., description="file path for local file")
    questions: List[str] = Field(..., description="list of questions")

# Retrieval-based QA function
def retrival_qa(questions: List[str], vector_store, top_k=5):
  """
    Retrieves relevant documents and generates answers using a language model.

    Args:
        questions (List[str]): List of questions to answer.
        vector_store (InMemoryVectorStore): The vector store containing indexed pages.
        top_k (int): Number of documents to retrieve for each question.

    Returns:
        dict: A dictionary of questions and their corresponding answers.
  """
  answers = {}
  for que in questions:
    docs = vector_store.similarity_search(que, k=5)
    # Combine the content of the retrieved documents
    context = "\n\n".join([doc.page_content for doc in docs])

    # Create a prompt for the language model
    prompt = f"Use the following context to answer the question:\n\n{context}\n\nQuestion: {que}\nAnswer:"

    # Generate the answer using the language model
    answer = llm(prompt)
    answers[que] = answer

  return answers


# Custom tool for retrieval-based QA
class RetrivalQA(BaseTool):
    """
    A custom tool that answers questions using a retrieval-based QA system.

    Attributes:
        name (str): Name of the tool.
        description (str): Description of the tool's functionality.
        args_schema (Type[BaseModel]): Input schema defining the required arguments.
        return_direct (bool): Whether to return the result directly or not.
    """
    name:str = "retrieval_qa"
    description:str = "Answers questions using a retrieval-based QA system."
    args_schema: Type[BaseModel] = UserInput
    return_direct: bool = True

    def _run(
        self,
        file_path: str,
        questions: List[str],
    ) -> Union[str, Any]:
        """Runs the retrieval-based question answering for the list of questions."""
        try:
            pages = pdf_loader(file_path)
            vector_store = create_vector_store(pages = pages)
            # Call the retrieval_qa function and get answers
            answers = retrival_qa(questions=questions, vector_store=vector_store)
            return str(answers)  # Return the answers as a string
        except Exception as e:
            return f"Error during retrieval: {str(e)}"
retrival_tool = RetrivalQA()

tools = [retrival_tool]



In [86]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import AIMessage, HumanMessage

# agent creater
agent_executor = create_react_agent(llm, tools)

In [87]:
agent_executor.invoke(
    {
        "messages": [
            HumanMessage(content="""File_Path:/content/handbook.pdf
    Qusetions:
    What is the name of the company?
    Who is the CEO of the company?
    What is their vacation policy?
    What is the termination policy?
  """
  )]}
)

{'messages': [HumanMessage(content='File_Path:/content/handbook.pdf\n    Qusetions:\n    What is the name of the company?\n    Who is the CEO of the company?\n    What is their vacation policy?\n    What is the termination policy?\n  ', additional_kwargs={}, response_metadata={}, id='d529c116-d375-4b32-a5ad-ecb7d110b94f'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_P7Wq1srRhEv2TjnGlVewUl2P', 'function': {'arguments': '{"file_path":"/content/handbook.pdf","questions":["What is the name of the company?","Who is the CEO of the company?","What is their vacation policy?","What is the termination policy?"]}', 'name': 'retrieval_qa'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 110, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_t

In [88]:
"""
{'messages': [HumanMessage(content='File_Path:/content/handbook.pdf\n    Qusetions:\n    What is the name of the company?\n    Who is the CEO of the company?\n    What is their vacation policy?\n    What is the termination policy?\n  ', additional_kwargs={}, response_metadata={}, id='d529c116-d375-4b32-a5ad-ecb7d110b94f'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_P7Wq1srRhEv2TjnGlVewUl2P', 'function': {'arguments': '{"file_path":"/content/handbook.pdf","questions":["What is the name of the company?","Who is the CEO of the company?","What is their vacation policy?","What is the termination policy?"]}', 'name': 'retrieval_qa'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 110, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2f41cae4-3084-4f53-b090-faeb6b0ed2f8-0', tool_calls=[{'name': 'retrieval_qa', 'args': {'file_path': '/content/handbook.pdf', 'questions': ['What is the name of the company?', 'Who is the CEO of the company?', 'What is their vacation policy?', 'What is the termination policy?']}, 'id': 'call_P7Wq1srRhEv2TjnGlVewUl2P', 'type': 'tool_call'}], usage_metadata={'input_tokens': 110, 'output_tokens': 59, 'total_tokens': 169, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  ToolMessage(content='{\'What is the name of the company?\': AIMessage(content=\'The name of the company is Zania, Inc.\', additional_kwargs={\'refusal\': None}, response_metadata={\'token_usage\': {\'completion_tokens\': 12, \'prompt_tokens\': 2120, \'total_tokens\': 2132, \'completion_tokens_details\': {\'accepted_prediction_tokens\': 0, \'audio_tokens\': 0, \'reasoning_tokens\': 0, \'rejected_prediction_tokens\': 0}, \'prompt_tokens_details\': {\'audio_tokens\': 0, \'cached_tokens\': 1920}}, \'model_name\': \'gpt-4o-mini-2024-07-18\', \'system_fingerprint\': \'fp_d02d531b47\', \'finish_reason\': \'stop\', \'logprobs\': None}, id=\'run-f63328da-9313-4097-b461-a398c64ab2dc-0\', usage_metadata={\'input_tokens\': 2120, \'output_tokens\': 12, \'total_tokens\': 2132, \'input_token_details\': {\'audio\': 0, \'cache_read\': 1920}, \'output_token_details\': {\'audio\': 0, \'reasoning\': 0}}), \'Who is the CEO of the company?\': AIMessage(content=\'The CEO of the company is Shruti Gupta.\', additional_kwargs={\'refusal\': None}, response_metadata={\'token_usage\': {\'completion_tokens\': 11, \'prompt_tokens\': 1958, \'total_tokens\': 1969, \'completion_tokens_details\': {\'accepted_prediction_tokens\': 0, \'audio_tokens\': 0, \'reasoning_tokens\': 0, \'rejected_prediction_tokens\': 0}, \'prompt_tokens_details\': {\'audio_tokens\': 0, \'cached_tokens\': 1792}}, \'model_name\': \'gpt-4o-mini-2024-07-18\', \'system_fingerprint\': \'fp_d02d531b47\', \'finish_reason\': \'stop\', \'logprobs\': None}, id=\'run-d007138e-c729-4d31-b470-8c8d6b0228cf-0\', usage_metadata={\'input_tokens\': 1958, \'output_tokens\': 11, \'total_tokens\': 1969, \'input_token_details\': {\'audio\': 0, \'cache_read\': 1792}, \'output_token_details\': {\'audio\': 0, \'reasoning\': 0}}), \'What is their vacation policy?\': AIMessage(content="Zania, Inc. provides paid vacation to its employees, with the following key points regarding their vacation policy:\\n\\n1. **Eligibility**: All full-time regular employees are eligible for vacation time, which can be available immediately upon hire, after completing an introductory period, or after completing a certain number of days of employment, depending on the company\'s specific policy.\\n\\n2. **Vacation Accrual**: Vacation is calculated based on either the employee\'s work anniversary year, the calendar year, or the fiscal year. Employees may accrue vacation according to a predetermined schedule based on their length of service, with different amounts granted for each year of employment.\\n\\n3. **Maximum Accrual**: There is a maximum accrual limit for vacation time. Once employees reach this limit, they will not accrue additional vacation until they use some of their accrued time.\\n\\n4. **Leave Usage**: Employees are encouraged to use their vacation time. Requests for vacation should be made to a manager as far in advance as possible, with a specified notice period required. The company aims to grant vacation requests, considering business needs and employee seniority.\\n\\n5. **Carryover**: Depending on the company\'s policy, unused vacation may carry over to the following year, or it may be forfeited if not used by a certain date.\\n\\n6. **Separation of Employment**: Upon leaving the company, employees may either forfeit any earned but unused vacation time or be paid for it, unless dictated otherwise by state law.\\n\\n7. **During Leave of Absence**: Employees may be required to use any unused vacation during disability or family medical leave, where permissible by law.\\n\\nOverall, the policy emphasizes the importance of taking time off while providing a structure for accrual, usage, and carryover of vacation time.", additional_kwargs={\'refusal\': None}, response_metadata={\'token_usage\': {\'completion_tokens\': 359, \'prompt_tokens\': 2911, \'total_tokens\': 3270, \'completion_tokens_details\': {\'accepted_prediction_tokens\': 0, \'audio_tokens\': 0, \'reasoning_tokens\': 0, \'rejected_prediction_tokens\': 0}, \'prompt_tokens_details\': {\'audio_tokens\': 0, \'cached_tokens\': 2688}}, \'model_name\': \'gpt-4o-mini-2024-07-18\', \'system_fingerprint\': \'fp_0aa8d3e20b\', \'finish_reason\': \'stop\', \'logprobs\': None}, id=\'run-4155236e-1dd4-410b-a310-b6e1dd96bfce-0\', usage_metadata={\'input_tokens\': 2911, \'output_tokens\': 359, \'total_tokens\': 3270, \'input_token_details\': {\'audio\': 0, \'cache_read\': 2688}, \'output_token_details\': {\'audio\': 0, \'reasoning\': 0}}), \'What is the termination policy?\': AIMessage(content=\'The termination policy at Zania, Inc. is based on "at-will" employment, which means that either the employee or the Company can terminate the employment relationship at any time, with or without notice and with or without cause. This policy does not create a contract for guaranteed or continued employment, nor does it provide rights to termination only "for cause." The CEO is the only person authorized to make promises regarding guaranteed or continued employment, and such promises must be in writing and signed by the CEO. Overall, the Company reserves the right to discipline or terminate employees for any reason permitted by law.\', additional_kwargs={\'refusal\': None}, response_metadata={\'token_usage\': {\'completion_tokens\': 121, \'prompt_tokens\': 3287, \'total_tokens\': 3408, \'completion_tokens_details\': {\'accepted_prediction_tokens\': 0, \'audio_tokens\': 0, \'reasoning_tokens\': 0, \'rejected_prediction_tokens\': 0}, \'prompt_tokens_details\': {\'audio_tokens\': 0, \'cached_tokens\': 0}}, \'model_name\': \'gpt-4o-mini-2024-07-18\', \'system_fingerprint\': \'fp_0aa8d3e20b\', \'finish_reason\': \'stop\', \'logprobs\': None}, id=\'run-3dd7dad9-72be-415a-afa9-b3274dcc2c10-0\', usage_metadata={\'input_tokens\': 3287, \'output_tokens\': 121, \'total_tokens\': 3408, \'input_token_details\': {\'audio\': 0, \'cache_read\': 0}, \'output_token_details\': {\'audio\': 0, \'reasoning\': 0}})}', name='retrieval_qa', id='f046b96a-9851-4d13-8369-eb5e6ae90bbc', tool_call_id='call_P7Wq1srRhEv2TjnGlVewUl2P')]}
"""

'\n{\'messages\': [HumanMessage(content=\'File_Path:/content/handbook.pdf\n    Qusetions:\n    What is the name of the company?\n    Who is the CEO of the company?\n    What is their vacation policy?\n    What is the termination policy?\n  \', additional_kwargs={}, response_metadata={}, id=\'d529c116-d375-4b32-a5ad-ecb7d110b94f\'),\n  AIMessage(content=\'\', additional_kwargs={\'tool_calls\': [{\'id\': \'call_P7Wq1srRhEv2TjnGlVewUl2P\', \'function\': {\'arguments\': \'{"file_path":"/content/handbook.pdf","questions":["What is the name of the company?","Who is the CEO of the company?","What is their vacation policy?","What is the termination policy?"]}\', \'name\': \'retrieval_qa\'}, \'type\': \'function\'}], \'refusal\': None}, response_metadata={\'token_usage\': {\'completion_tokens\': 59, \'prompt_tokens\': 110, \'total_tokens\': 169, \'completion_tokens_details\': {\'accepted_prediction_tokens\': 0, \'audio_tokens\': 0, \'reasoning_tokens\': 0, \'rejected_prediction_tokens\': 0}, \'